## Research Project 4
---
```text
- Source: SEC
- Goal: Extract CEO appointments from 8K
- Techniques: sentence tokenization
- Tools: NLTK
- Lines of code: ~50 (incomplete)```

### Request and parse SEC index
---

In [6]:
# Let's find a page on the SEC website with all 8-Ks
url = 'https://www.sec.gov/Archives/edgar/daily-index/2018/QTR2/form.20180402.idx'

In [7]:
import requests
res = requests.get(url)

<div class="alert alert-block alert-info" 
     style="border-color: #2E86C1;
            background-color: #FCF3CF;
            border-left: 5px solid #2E86C1;
            padding-top: 5px">
    <font size="4">
    <a href="../deep_dives/http.ipynb" style="text-decoration: none"> 
    <h3 style="font-family: monospace">Deep-dive</h3>
    <p style="margin-left: 100px;
              margin-right: 100px;
              line-height: 1.7em;">Hypertext Transfer Protocol (HTTP)</p></a></font>
</div>

In [8]:
print(res.text[:1000])

Description:           Daily Index of EDGAR Dissemination Feed by Form Type
Last Data Received:    Apr  2, 2018
Comments:              webmaster@sec.gov
Anonymous FTP:         ftp://ftp.sec.gov/edgar/
 
 
 
 
Form Type   Company Name                                                  CIK
      Date Filed  File Name
---------------------------------------------------------------------------------------------------------------------------------------------
1-SA        HYGEN INDUSTRIES, INC.                                        1661116     20180402    edgar/data/1661116/0001065949-18-000055.txt         
1-U         FUNDRISE REAL ESTATE INVESTMENT TRUST, LLC                    1645583     20180402    edgar/data/1645583/0001144204-18-018374.txt         
1-U         Fundrise East Coast Opportunistic REIT, LLC                   1660918     20180402    edgar/data/1660918/0001144204-18-018370.txt         
1-U         Fundrise Equity REIT, LLC                                     1648956     2018

In [9]:
line = '1-SA        HYGEN INDUSTRIES, INC.                                        1661116     20180402    edgar/data/1661116/0001065949-18-000055.txt'

In [15]:
print(line.find('HYGEN')); 
print(line.find('1661116')); 
print(line.find('20180402')); 
print(line.find('edgar/data'))

12
74
86
98


In [18]:
record = {
    'Form Type': line[0:12].strip(),
    'Company Name': line[12:74].strip(),
    'CIK': line[74:86].strip(),
    'Date Filed': line[86:98].strip(),
    'File Name': line[98:].strip()
}
record

{'CIK': '1661116',
 'Company Name': 'HYGEN INDUSTRIES, INC.',
 'Date Filed': '20180402',
 'File Name': 'edgar/data/1661116/0001065949-18-000055.txt',
 'Form Type': '1-SA'}

<div class="alert alert-block alert-info" 
     style="border-color: #2E86C1; 
            border-left: 5px solid #2E86C1;
            padding-top: 5px">
    <font size="4"> 
    <a href="./playground.ipynb" style="text-decoration: none"> 
    <h3 style="font-family: monospace">Exercise 4.1</h3>
    <p style="margin-left: 100px;
              margin-right: 100px;
              line-height: 1.7em;">Create a function that parses "Date Filed" into a python datetime object.
    </p></a></font>
</div>

<div class="alert alert-block alert-info" 
     style="border-color: #2E86C1; 
            border-left: 5px solid #2E86C1;
            padding-top: 5px">
    <font size="4"> 
    <a href="./playground.ipynb" style="text-decoration: none"> 
    <h3 style="font-family: monospace">Exercise 4.2</h3>
    <p style="margin-left: 100px;
              margin-right: 100px;
              line-height: 1.7em;">Create a function that, given an SEC index, parses it and returns a list of records.
    </p></a></font>
</div>

### Parse 8-K
---

In [22]:
for line in res.text.split('\n')[11:]:
    if line.startswith('8-K'):
        url = 'https://www.sec.gov/Archives/' + line[98:].strip()
        doc = requests.get(url)
        break

In [25]:
url

'https://www.sec.gov/Archives/edgar/data/318306/0001144204-18-018693.txt'

In [24]:
doc.text[:1000]

'<SEC-DOCUMENT>0001144204-18-018693.txt : 20180402\n<SEC-HEADER>0001144204-18-018693.hdr.sgml : 20180402\n<ACCEPTANCE-DATETIME>20180402170558\nACCESSION NUMBER:\t\t0001144204-18-018693\nCONFORMED SUBMISSION TYPE:\t8-K\nPUBLIC DOCUMENT COUNT:\t\t3\nCONFORMED PERIOD OF REPORT:\t20180329\nITEM INFORMATION:\t\tDeparture of Directors or Certain Officers; Election of Directors; Appointment of Certain Officers: Compensatory Arrangements of Certain Officers\nITEM INFORMATION:\t\tRegulation FD Disclosure\nITEM INFORMATION:\t\tFinancial Statements and Exhibits\nFILED AS OF DATE:\t\t20180402\nDATE AS OF CHANGE:\t\t20180402\n\nFILER:\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tABEONA THERAPEUTICS INC.\n\t\tCENTRAL INDEX KEY:\t\t\t0000318306\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:\tPHARMACEUTICAL PREPARATIONS [2834]\n\t\tIRS NUMBER:\t\t\t\t830221517\n\t\tSTATE OF INCORPORATION:\t\t\tDE\n\t\tFISCAL YEAR END:\t\t\t1231\n\n\tFILING VALUES:\n\t\tFORM TYPE:\t\t8-K\n\t\tSEC ACT:\t\t1934 Act\n

In [28]:
from lxml import html
tree = html.fromstring(doc.content)

In [30]:
' '.join(tree.itertext())[:1000]

'0001144204-18-018693.txt : 20180402\n 0001144204-18-018693.hdr.sgml : 20180402\n 20180402170558\nACCESSION NUMBER:\t\t0001144204-18-018693\nCONFORMED SUBMISSION TYPE:\t8-K\nPUBLIC DOCUMENT COUNT:\t\t3\nCONFORMED PERIOD OF REPORT:\t20180329\nITEM INFORMATION:\t\tDeparture of Directors or Certain Officers; Election of Directors; Appointment of Certain Officers: Compensatory Arrangements of Certain Officers\nITEM INFORMATION:\t\tRegulation FD Disclosure\nITEM INFORMATION:\t\tFinancial Statements and Exhibits\nFILED AS OF DATE:\t\t20180402\nDATE AS OF CHANGE:\t\t20180402\n\nFILER:\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tABEONA THERAPEUTICS INC.\n\t\tCENTRAL INDEX KEY:\t\t\t0000318306\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:\tPHARMACEUTICAL PREPARATIONS [2834]\n\t\tIRS NUMBER:\t\t\t\t830221517\n\t\tSTATE OF INCORPORATION:\t\t\tDE\n\t\tFISCAL YEAR END:\t\t\t1231\n\n\tFILING VALUES:\n\t\tFORM TYPE:\t\t8-K\n\t\tSEC ACT:\t\t1934 Act\n\t\tSEC FILE NUMBER:\t001-15771\n\t\tFILM NUM

In [37]:
import re
text = ' '.join(tree.itertext())
text = re.sub(r'\n|\t|\xa0', ' ', text)
text = re.sub(' +', ' ', text)

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/marco/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from nltk.tokenize import sent_tokenize
char = text.find('Item 5.02')
if char >= 0:
    sentences = sent_tokenize(text[char:])
    joined = ' '.join(sentences[0:5])
    print(joined)

[nltk_data] Downloading package punkt to /Users/marco/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


NameError: name 'text' is not defined

This requires building a training set for text classification, likely with a deep neural network. 

We'll stop here, but here's a possible solution.

1. Generate a training set including sentences from `Item 5.02`. 
1. Label each sample 1 if it represents a `CEO turnover`, 0 otherwise (or more granular if other events are of interest)
1. Replace every word with its embedding (`word2vec`, or `sentence encoder`)
1. Train a Long-Short Term Memory (Recurrent Neural Network) on the dataset

See for example [here](https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/)